<a href="https://colab.research.google.com/github/p4guiler4/Reinforcement_Learning/blob/master/Reinforcement_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement learning

In a way, Reinforcement Learning is the science of making optimal decisions using experiences. Breaking it down, the process of Reinforcement Learning involves these simple steps:

1. Observation of the environment
2. Deciding how to act using some strategy
3. Acting accordingly
4. Receiving a reward or penalty
5. Learning from the experiences and refining our strategy
6. Iterate until an optimal strategy is found

![The Reinforcement Learning Process](https://www.learndatasci.com/documents/14/Reinforcement-Learning-Animation.gif)

In [0]:
!pip install cmake 'gym[atari]' scipy

# Gym's interface

Gym provides different game environments which we can plug into our code and test an agent. The library takes care of API for providing all the information that our agent would require, like possible actions, score, and current state. We just need to focus just on the algorithm part for our agent.

We'll be using the Gym environment called Taxi-V2, which all of the details explained above were pulled from. The objectives, rewards, and actions are all the same.

In [0]:
import gym

env = gym.make("Taxi-v2").env

env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



The core gym interface is env, which is the unified environment interface. 

# The Problem

"There are 4 locations (labeled by different letters), and our job is to pick up the passenger at one location and drop him off at another. We receive +20 points for a successful drop-off and lose 1 point for every time-step it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions."

In [0]:
env.reset()

env.render()

print("Action Space {}".format(env.action_space)) # 6 actions: 4 directions + pickup + dropoff
print("State Space {}".format(env.observation_space)) # taxi + passenger + destination locations

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


* The filled square represents the taxi, which is yellow without a passenger and green with a passenger.

* The pipe ("|") represents a wall which the taxi cannot cross.

* R, G, Y, B are the possible pickup and destination locations. The blue letter represents the current passenger pick-up location, and the purple letter is the current destination.





When we also account for one (1) additional passenger state of being inside the taxi, we can take all combinations of passenger locations and destination locations to come to a total number of states for our taxi environment; there's four (4) destinations and five (4 + 1) passenger locations.

So, our taxi environment has 5 (grid height, taxi Y) ×5 (grid width, taxi X) ×5 (passengers location, including inside taxi) ×4 (destination) = **500 total possible states.**

Reinforcement Learning will learn a mapping of **states** to the optimal **action** to perform in that state by *exploration*, i.e. the agent explores the environment and takes actions based off rewards defined in the environment.

The optimal action for each state is the action that has the **highest cumulative long-term reward.**

In [0]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)

print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



# The Reward Table

When the Taxi environment is created, there is an initial **Reward table** that's also created, called `P`. We can think of it like a matrix that has the number of states as rows and number of actions as columns, i.e. a states × actions matrix.

Since every state is in this matrix, we can see the default reward values assigned to our illustration's state:

In [0]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

**This dictionary has the structure {action: [(probability, nextstate, reward, done)]}.**

A few things to note:

* The 0-5 corresponds to the actions (south, north, east, west, pickup, dropoff) the taxi can perform at our current state in the illustration.

* In this env, probability is always 1.0.

* The nextstate is the state we would be in if we take the action at this index of the dict

* All the movement actions have a -1 reward and the pickup/dropoff actions have -10 reward in this particular state. If we are in a state where the taxi has a passenger and is on top of the right destination, we would see a reward of 20 at the dropoff action (5)

* done is used to tell us when we have successfully dropped off a passenger in the right location. Each successfull dropoff is the end of an episode

Note that if our agent chose to explore action three (3) in this state it would be going West into a wall. The source code has made it impossible to actually move the taxi across a wall, so if the taxi chooses that action, it will just keep accruing -1 penalties, which affects the long-term reward.

# Solving the environment with Brute Force

Since we have our P table for default rewards in each state, we can try to have our taxi navigate just using that.

We'll create an infinite loop which runs until one passenger reaches one destination (one episode), or in other words, when the received reward is 20. The env.action_space.sample() method automatically selects one random action from set of all possible actions.

In [0]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    
    # Ojo, la lógica del movimiento del taxi y de coger y soltar pasajeros está implementada en AIGym. En nuestro caso, estará implementada por escrituras y lecturas en el CHT_CLI.

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 651
Penalties incurred: 212


In [0]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 651
State: 0
Action: 5
Reward: 20


Not good. Our agent takes thousands of timesteps and makes lots of wrong drop offs to deliver just one passenger to the right destination.

This is because we aren't learning from past experience. We can run this over and over, and it will never optimize. The agent has no memory of which action was best for each state, which is exactly what Reinforcement Learning will do for us.

# Q-Learning (Quality Learning)

Essentially, Q-learning lets the agent use the environment's rewards to learn, over time, the best action to take in a given state.

In our Taxi environment, we have the *reward table, P*, that the agent will learn from. It does thing by looking receiving a reward for taking an action in the current state, then updating a Q-value **to remember if that action was beneficial**.

The values store in the *Q-table* are called a Q-values, and they map to a **(state, action)** combination.

A Q-value for a particular state-action combination **is representative of the "quality" of an action taken from that state**. Better Q-values imply better chances of getting greater rewards.

For example, if the taxi is faced with a state that includes a passenger at its current location, it is highly likely that the Q-value for pickup is higher when compared to other actions, like dropoff or north.

Q-values are initialized to an arbitrary value, and **as the agent exposes itself to the environment and receives different rewards by executing different actions**, the Q-values are updated using the equation:

![texto alternativo](https://wikimedia.org/api/rest_v1/media/math/render/svg/59db58edf1222b292e40706e503ed5974553606b)

Where:

- α (alpha) is the learning rate (0<α≤1) - Just like in supervised learning settings, α is the extent to which our Q-values are being updated in every iteration.

- γ (gamma) is the discount factor (0≤γ≤1) - determines how much importance we want to give to future rewards. A high value for the discount factor (close to 1) captures the long-term effective award, whereas, a discount factor of 0 makes our agent consider only immediate reward, hence making it greedy.

γ = 0 (cigarra)
γ  = 1 (hormiga)

# What is this saying?

We are assigning (←), or updating, the Q-value of the agent's current state and action by first taking a weight (1−α) of the old Q-value, then adding the learned value. The learned value is a combination of the reward for taking the current action in the current state (from the Reward Table P), and the discounted maximum reward from the next state we will be in once we take the current action.

Basically, we are learning the proper action to take in the current state by looking at the reward for the current state/action combo, and the max rewards for the next state. This will eventually cause our taxi to consider the route with the best rewards strung together.

The Q-value of a state-action pair is the sum of the instant reward and the discounted future reward (of the resulting state). The way we store the Q-values for each state and action is through a Q-table.

Entrenamiento:

Q <- Q + (recompensa inmediata + recompensa futura si sigo esta estrategia)

# Q-Table

The Q-table is a matrix where we have a row for every state (500) and a column for every action (6). It's first initialized to 0, and then values are updated after training. Note that the Q-table has the same dimensions as the reward table, but it has a completely different purpose.

![texto alternativo](https://upload.wikimedia.org/wikipedia/commons/e/e0/Q-Learning_Matrix_Initialized_and_After_Training.png)

# Summing up the Q-Learning Process

Breaking it down into steps, we get

* Initialize the Q-table by all zeros.

* Start exploring actions: For each state, select any one among all possible actions for the current state (S).

* Travel to the next state (S') as a result of that action (a).

* For all possible actions from the state (S') select the one with the highest Q-value.

* Update Q-table values using the equation.

* Set the next state as the current state.

* If goal state is reached, then end and repeat the process.

# Exploiting learned values

After enough random exploration of actions, the Q-values tend to converge serving our agent as an action-value function which it can exploit to pick the most optimal action from a given state.

There's a tradeoff between exploration (choosing a random action) and exploitation (choosing actions based on already learned Q-values). We want to prevent the action from always taking the same route, and possibly overfitting, so we'll be introducing another parameter called ϵ "epsilon" to cater to this during training.

**Instead of just selecting the best learned Q-value action, we'll sometimes favor exploring the action space further.** Lower epsilon value results in episodes with more penalties (on average) which is obvious because we are exploring and making random decisions.

# Training the Agent

Initialize the Q-Table

In [0]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

We can now create the training algorithm that will update this Q-table as the agent explores the environment over thousands of episodes.

In the first part of while not done, we decide whether to pick a random action or to exploit the already computed Q-values. This is done simply by using the epsilon value and comparing it to the random.uniform(0, 1) function, which returns an arbitrary number between 0 and 1.

We execute the chosen action in the environment to obtain the next_state and the reward from performing the action. After that, we calculate the maximum Q-value for the actions corresponding to the next_state, and with that, we can easily update our Q-value to the new_q_value:

In [0]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space. Me muevo al azar por explorar
        else:
            action = np.argmax(q_table[state]) # Exploit learned values. Tomo la mejor estrategia según lo que aprendí

        next_state, reward, done, info = env.step(action) # Observo qué ha pasado por mi acción. He sido castigado o premiado y tengo un nuevo estado
        
            # Ojo, la lógica del movimiento del taxi y de coger y soltar pasajeros está implementada en AIGym. En nuestro caso, estará implementada por escrituras y lecturas en el CHT_CLI
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max) # Fórmula. Calculo la calidad de esta estrategia a corto y medio plazo
        q_table[state, action] = new_value # Aprendo la estrategia

        if reward == -10:
            penalties += 1

        state = next_state # actualizo el estado (y por tanto el entorno)
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Now that the Q-table has been established over 100,000 episodes, let's see what the Q-values are at our illustration's state:

In [0]:
q_table[328]

The max Q-value is "north" (-1.971), so it looks like Q-learning has effectively learned the best action to take in our illustration's state!

#Evaluating the agent

Let's evaluate the performance of our agent. We don't need to explore actions any further, so now the next action is always selected using the best Q-value:

In [0]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

frames = []

for ep in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
                    
        # Put each rendered frame into dict for animation
        
        frames.append({
         'frame': env.render(mode='ansi'),
         'state': state,
         'action': action,
         'reward': reward,
         'episode': ep
         }
        )

        if reward == -10:
            penalties += 1

        epochs += 1
        
    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.7
Average penalties per episode: 0.0


In [0]:

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print(f"EPISODE: {frame['episode']}")
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

EPISODE: 99
Timestep: 1270
State: 85
Action: 5
Reward: 20


We can see from the evaluation, the agent's performance improved significantly and it incurred no penalties, which means it performed the correct pickup/dropoff actions with 100 different passengers.